<a href="https://colab.research.google.com/github/srirampai/Active_learning/blob/main/Banking_Keras_2nd_march.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [110]:
import tensorflow_datasets as tfds
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import matplotlib.pyplot as plt
import re
import string
import numpy as np
import logging
import keras
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout 
from keras.wrappers.scikit_learn import KerasClassifier
import pandas as pd

In [113]:
train_data = pd.read_csv('Train.csv')
#test_data= pd.read_csv('Test.csv')
#dimensions of the data
print((train_data.shape))
#print((test_data.shape))

(10003, 2)


In [114]:
train_data.head()

,text,label
0,I am still waiting on my card?,card_arrival
1,What can I do if my card still hasn't arrived ...,card_arrival
2,I have been waiting over a week. Is the card s...,card_arrival
3,Can I track my card while it is in the process...,card_arrival
4,"How do I know if I will get my card, or if it ...",card_arrival


**Lets check for any null entries**

In [115]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10003 entries, 0 to 10002
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   text    10003 non-null  object
 1   label   10003 non-null  object
dtypes: object(2)
memory usage: 156.4+ KB


In [116]:
test_data.head()

,text,label
0,How do I locate my card?,card_arrival
1,"I still have not received my new card, I order...",card_arrival
2,I ordered a card but it has not arrived. Help ...,card_arrival
3,Is there a way to know when my card will arrive?,card_arrival
4,My card has not arrived yet.,card_arrival


**High level Describe**

In [117]:
test_data.describe()

,text,label
count,3080,3080
unique,3080,77
top,How do I locate my card?,card_arrival
freq,1,40


In [118]:
train_data.describe()

,text,label
count,10003,10003
unique,10003,77
top,I am still waiting on my card?,card_payment_fee_charged
freq,1,187


In [119]:
num_classes = np.unique(train_data['label']).shape[0]
print("the number of unique output classes are:", num_classes)

the number of unique output classes are: 77


In [120]:
output_classes_array =train_data.label.unique()
output_classes_array

array(['card_arrival', 'card_linking', 'exchange_rate',
       'card_payment_wrong_exchange_rate', 'extra_charge_on_statement',
       'pending_cash_withdrawal', 'fiat_currency_support',
       'card_delivery_estimate', 'automatic_top_up', 'card_not_working',
       'exchange_via_app', 'lost_or_stolen_card', 'age_limit',
       'pin_blocked', 'contactless_not_working',
       'top_up_by_bank_transfer_charge', 'pending_top_up',
       'cancel_transfer', 'top_up_limits',
       'wrong_amount_of_cash_received', 'card_payment_fee_charged',
       'transfer_not_received_by_recipient',
       'supported_cards_and_currencies', 'getting_virtual_card',
       'card_acceptance', 'top_up_reverted',
       'balance_not_updated_after_cheque_or_cash_deposit',
       'card_payment_not_recognised', 'edit_personal_details',
       'why_verify_identity', 'unable_to_verify_identity',
       'get_physical_card', 'visa_or_mastercard', 'topping_up_by_card',
       'disposable_card_limits', 'compromised_card

**Output Text Encoding**

In [121]:
from pandas.core.dtypes.cast import convert_dtypes
train_data['label'] =  train_data['label'].astype('category')
train_data['label_id'] = train_data['label'].cat.codes
train_data[100:200]


,text,label,label_id
100,can you share card tracking number?,card_arrival,12
101,I have not received my card and it's been a we...,card_arrival,12
102,Is there a way I can check on the card on rout...,card_arrival,12
103,When will my new card arrive?,card_arrival,12
104,Why haven't I received my new card yet?,card_arrival,12
...,...,...,...
195,I found my lost card. Am I still able to use it?,card_linking,14
196,Will I need a new card or can I put my old car...,card_linking,14
197,Is there a way to link the card i just got?,card_linking,14
198,How do I add a new card?,card_linking,14


In [ ]:
#test_data['label'] =  test_data['label'].astype('category')
#test_data['label_id'] = test_data['label'].cat.codes


In [ ]:
#test_data.info()

In [ ]:
train_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10003 entries, 0 to 10002
Data columns (total 3 columns):
 #   Column    Non-Null Count  Dtype   
---  ------    --------------  -----   
 0   text      10003 non-null  object  
 1   label     10003 non-null  category
 2   label_id  10003 non-null  int8    
dtypes: category(1), int8(1), object(1)
memory usage: 100.5+ KB


***Save the Label encoding as Dictioanry***

In [122]:
id_to_category = pd.Series(train_data.label.values, index = train_data.label_id).to_dict()
id_to_category

{0: 'Refund_not_showing_up',
 1: 'activate_my_card',
 2: 'age_limit',
 3: 'apple_pay_or_google_pay',
 4: 'atm_support',
 5: 'automatic_top_up',
 6: 'balance_not_updated_after_bank_transfer',
 7: 'balance_not_updated_after_cheque_or_cash_deposit',
 8: 'beneficiary_not_allowed',
 9: 'cancel_transfer',
 10: 'card_about_to_expire',
 11: 'card_acceptance',
 12: 'card_arrival',
 13: 'card_delivery_estimate',
 14: 'card_linking',
 15: 'card_not_working',
 16: 'card_payment_fee_charged',
 17: 'card_payment_not_recognised',
 18: 'card_payment_wrong_exchange_rate',
 19: 'card_swallowed',
 20: 'cash_withdrawal_charge',
 21: 'cash_withdrawal_not_recognised',
 22: 'change_pin',
 23: 'compromised_card',
 24: 'contactless_not_working',
 25: 'country_support',
 26: 'declined_card_payment',
 27: 'declined_cash_withdrawal',
 28: 'declined_transfer',
 29: 'direct_debit_payment_not_recognised',
 30: 'disposable_card_limits',
 31: 'edit_personal_details',
 32: 'exchange_charge',
 33: 'exchange_rate',
 34: 

In [ ]:
#[print(key,':',value) for key, value in id_to_category.items()]

In [ ]:
# from sklearn.preprocessing import LabelEncoder
# Y_Train, Y_Test = train_data['label'], test_data['label']
# encoder = LabelEncoder()
# Y_Train_ID = encoder.fit_transform(Y_Train)
# Y_Test_ID = encoder.fit_transform(Y_Test)

**Pool separation**

In [123]:
# assemble initial data
n_initial = 8000
pool_data = train_data.iloc[-2000:]
#training_data = train_data.iloc[:n_initial]

#training_data.shape


In [124]:
#df.iloc[-3:]
pool_data.shape


(2000, 3)

In [125]:
pool_data.head()

,text,label,label_id
8003,Where can I deposit cash to top up?,top_up_by_cash_or_cheque,58
8004,Is there an option to top up with a cheque?,top_up_by_cash_or_cheque,58
8005,"Help, I need to top up my account. Where do I ...",top_up_by_cash_or_cheque,58
8006,I'll send a check to top up my account,top_up_by_cash_or_cheque,58
8007,Is it possible for me to top up with a cheque?,top_up_by_cash_or_cheque,58


In [127]:
training_data = train_data.iloc[:8000]
training_data.shape

(8000, 3)

In [128]:
training_data

,text,label,label_id
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12
2,I have been waiting over a week. Is the card s...,card_arrival,12
3,Can I track my card while it is in the process...,card_arrival,12
4,"How do I know if I will get my card, or if it ...",card_arrival,12
...,...,...,...
7995,I wish to be able to top up with cash.,top_up_by_cash_or_cheque,58
7996,Where do I top up?,top_up_by_cash_or_cheque,58
7997,Do you accept cheques for top-up?,top_up_by_cash_or_cheque,58
7998,How do I use a check to top up?,top_up_by_cash_or_cheque,58


**Training data = Train Data + Pool data**

In [138]:
train_data = training_data
train_data

,text,label,label_id
0,I am still waiting on my card?,card_arrival,12
1,What can I do if my card still hasn't arrived ...,card_arrival,12
2,I have been waiting over a week. Is the card s...,card_arrival,12
3,Can I track my card while it is in the process...,card_arrival,12
4,"How do I know if I will get my card, or if it ...",card_arrival,12
...,...,...,...
7995,I wish to be able to top up with cash.,top_up_by_cash_or_cheque,58
7996,Where do I top up?,top_up_by_cash_or_cheque,58
7997,Do you accept cheques for top-up?,top_up_by_cash_or_cheque,58
7998,How do I use a check to top up?,top_up_by_cash_or_cheque,58


In [139]:
from sklearn.model_selection import train_test_split
features, targets = train_data['text'], train_data['label_id']
pool_features , pool_targets = pool_data['text'], pool_data['label_id']

train_features, test_features, train_targets, test_targets = train_test_split(
        features, targets,
        train_size=0.7,
        test_size=0.3,
        random_state=42,
        shuffle = True,
        stratify=targets
    )

In [140]:
train_features.values[:5]

array(['How long do I have to wait for Europe transfers?',
       "I can't remember my app code, can you reset it?",
       'I have some transactions on my account I did not make. Can you cancel my card and prevent further use.',
       'The app does not know it is me.', 'Can I use American Express?'],
      dtype=object)

In [141]:
train_targets.values[:5]

array([67, 45, 23, 68, 54], dtype=int8)

**Tensor Flow pipeline**
Prepare Tensor Flow pipeline

In [142]:
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
import os

import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras.layers import TextVectorization
import re
import string
import random
from sklearn.model_selection import train_test_split
import tensorflow as tf

from tensorflow import keras

In [143]:
# train X & y -- RAW still in string 
train_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(train_features.values, tf.string)
) 
train_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(train_targets.values, tf.int64),

) 
# test X & y
test_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(test_features.values, tf.string)
) 
test_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(test_targets.values, tf.int64),

) 

#pool X & Y
pool_text_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(pool_features.values, tf.string)
) 
pool_cat_ds_raw = tf.data.Dataset.from_tensor_slices(
            tf.cast(pool_targets.values, tf.int64),

) 


**Decide the dictionary size and the review size**

In [144]:
vocab_size = 20000  # Only consider the top 20K words
max_len = 50  # Maximum review (text) size in words

**Text Vectorisation**

In [145]:
def custom_standardization(input_data):
    lowercase = tf.strings.lower(input_data)
    stripped_html = tf.strings.regex_replace(lowercase, "<br />", " ")
    return tf.strings.regex_replace(
        stripped_html, f"[{re.escape(string.punctuation)}]", ""
    )

In [146]:
input_data = "Ive been trying to use my card for two weeks now to purchase somethings online but nothing is going through. It keeps returning my payment to my account. There shouldn't be anything wrong with my account, is it the card?"
output_dummy_string = custom_standardization(input_data)
print("output is:", output_dummy_string.numpy().decode("utf=8"))

output is: ive been trying to use my card for two weeks now to purchase somethings online but nothing is going through it keeps returning my payment to my account there shouldnt be anything wrong with my account is it the card


In [147]:
# VECTORISATION
vectorize_layer = layers.TextVectorization(
    3000, standardize=custom_standardization, output_sequence_length=150
)

In [160]:
# TRAIN DATA VECTORISATION & ADAPAT
train_data_Vectorised = vectorize_layer.adapt(train_features)
vocab = vectorize_layer.get_vocabulary()  # To get words back from token indices

In [161]:
# POOL DATA VECTORISATION & ADAPT
pool_data_Vectorised =vectorize_layer.adapt(pool_features)
pool_vocab = vectorize_layer.get_vocabulary()

In [150]:
print("vocab has the ", len(vocab)," entries")
print("vocab has the following first 10 entries")
for word in range(10):
  print(word, " represents the word: ", vocab[word])

for X in train_features[:2]:
  print(" Given raw data: " )
  print(X)
  tokenized = vectorize_layer(tf.expand_dims(X, -1))
  print(" Tokenized and Transformed to a vector of integers: " )
  print (tokenized)
  print(" Text after Tokenized and Transformed: ")
  transformed = ""
  for each in tf.squeeze(tokenized):
    transformed= transformed+ " "+ vocab[each]
  print(transformed)

vocab has the  1959  entries
vocab has the following first 10 entries
0  represents the word:  
1  represents the word:  [UNK]
2  represents the word:  i
3  represents the word:  my
4  represents the word:  to
5  represents the word:  the
6  represents the word:  a
7  represents the word:  card
8  represents the word:  is
9  represents the word:  can
 Given raw data: 
How long do I have to wait for Europe transfers?
 Tokenized and Transformed to a vector of integers: 
tf.Tensor(
[[ 13 100  10   2  32   5 331   9 499 169   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0
    0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   0   

In [ ]:
'''from sklearn.preprocessing import LabelEncoder
from keras.utils import np_utils
# encode class values as integers
Y = train_data['label']
encoder = LabelEncoder()
encoder.fit(Y)
encoded_Y = encoder.transform(Y)
# convert integers to dummy variables (i.e. one hot encoded)
dummy_y = np_utils.to_categorical(encoded_Y)
dummy_y'''

"from sklearn.preprocessing import LabelEncoder\nfrom keras.utils import np_utils\n# encode class values as integers\nY = train_data['label']\nencoder = LabelEncoder()\nencoder.fit(Y)\nencoded_Y = encoder.transform(Y)\n# convert integers to dummy variables (i.e. one hot encoded)\ndummy_y = np_utils.to_categorical(encoded_Y)\ndummy_y"

**MODEL LSTM KERAS**

In [151]:
# build function for the Keras' scikit-learn API
def create_model():
    model = keras.models.Sequential(
        [
            layers.Input(shape=(150,)),
            layers.Embedding(input_dim=3000, output_dim=128),
            layers.Bidirectional(layers.LSTM(32, return_sequences=True)),
            layers.GlobalMaxPool1D(),
            layers.Dense(20, activation="relu"),
            layers.Dropout(0.5),
            layers.Dense(1, activation="sigmoid"),
        ]
    )
    model.summary()
    return model


In [153]:
!pip install modAL

**Training the Active Leaner**

In [156]:
pip install scikeras[tensorflow]

     |████████████████████████████████| 462 kB 16.5 MB/s 
     |████████████████████████████████| 97 kB 5.5 MB/s 
  Attempting uninstall: importlib-metadata
    Found existing installation: importlib-metadata 4.11.1
    Uninstalling importlib-metadata-4.11.1:
      Successfully uninstalled importlib-metadata-4.11.1
  Attempting uninstall: markdown
    Found existing installation: Markdown 3.3.6
    Uninstalling Markdown-3.3.6:
      Successfully uninstalled Markdown-3.3.6


In [158]:
#from keras.wrappers.scikit_learn import KerasClassifier
from scikeras.wrappers import KerasClassifier, KerasRegressor
from modAL.models import ActiveLearner

In [159]:
# create the classifier
classifier = KerasClassifier(create_model)

In [162]:
# initialize ActiveLearner
learner = ActiveLearner(
    estimator=classifier,
    X_training=train_data_Vectorised, y_training= train_cat_ds_raw,
    verbose=1
)

**active learning**

In [163]:
# the active learning loop
n_queries = 10
for idx in range(n_queries):
    query_idx, query_instance = learner.query(train_data_Vectorised, n_instances=100, verbose=0)
    print(query_idx)
    learner.teach(
        X=pool_data_Vectorised[query_idx], y=pool_cat_ds_raw[query_idx], only_new=True,
        verbose=1
    )
    # remove queried instance from pool
    pool_text_ds_raw = np.delete(pool_text_ds_raw, query_idx, axis=0)
    pool_cat_ds_raw = np.delete(pool_cat_ds_raw, query_idx, axis=0)

AttributeError: ignored